<br><br><br><br><br>

# Accelerating Python

<br><br><br><br><br>

<br>

<p style="font-size: 1.25em">Apart from clever algorithms, the speed of a program is determined by how laden it is with work unrelated to its main task.</p>

<center><img src="img/swallows-coconut.jpg" width="50%"></center>

<p style="font-size: 1.25em">Python is slower than C because of dynamic type checking, garbage collection, everything-is-a-hashtable, pointer chasing, string equality checks...</p>

<br>

<p style="font-size: 1.25em">Compare to Java, which has garbage collection but not dynamic type checking, and C, which has neither (on a variety of benchmark algorithms).</p>

<img src="img/benchmark-games.png" width="100%">

<br><br>

<p style="font-size: 1.25em">If you care about speed <i>more than</i> ease of development, use C or C++ (or Rust!).</p>

<br>

<p style="font-size: 1.25em">If you want to use Python for all of its conveniences—dynamic type checking, garbage collection, everything-is-a-hashtable, interactive development and debugging—and need to speed up a critical section, there are ways to do it.</p>

<br>

<p style="font-size: 1.25em">But you have to give up Python's dynamism <i>in that section.</i></p>

<br><br>

In [1]:
# Some wacky things you can do to types and objects in Python:
class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y
p = Point(1, 2)

# dynamically add an attribute to an instance (attributes are really a hashtable)
p.z = 3

# dynamically add a method to a class (class is a hashtable of functions)
Point.mag2 = lambda self: self.x**2 + self.y**2 + self.z**2
p.mag2()

# dynamically add a method to an instance (differs only in assigning "self")
p.mag = (lambda self: self.mag2()**0.5).__get__(p)
p.mag()

class Pointy(Point):
    def __repr__(self):
        return "Pointy({0}, {1})".format(self.x, self.y)

# dynamically change the class of an object (type is just an attribute)
p.__class__ = Pointy
p

Pointy(1, 2)

<br><br>

<p style="font-size: 1.25em">As a statically compiled program, Python has only one data type, <tt>PyObject*</tt> with a pointer to its runtime type, which is yet another <tt>PyObject*</tt>.</p>

<center><img src="img/pyobject.png" width="50%"></center>

<p style="font-size: 1.25em">That gives us the flexibility to do all the wacky things on the previous slide, but at a runtime cost of checking those types <i>every time they are used.</i></p>

<br><br>

In [2]:
# x is an integer
x = 0

for i in range(1000000):
    # in the millionth step, replace x with a string
    if i == 999999:
        x = "hello"
    # do an operation on x that only works for integers
    x += 1

TypeError: can only concatenate str (not "int") to str

In [3]:
import numba

@numba.jit(nopython=True)
def f():
    x = 0
    for i in range(1000000):
        # in the millionth step, replace x with a string
        if i == 999999:
            x = "hello"
        # do an operation on x that only works for integers
        x += 1
    return x

f()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Cannot unify int64 and Literal[str](hello) for 'x', defined at <ipython-input-3-290cb8971149> (5)

File "<ipython-input-3-290cb8971149>", line 5:
def f():
    x = 0
    ^

[1] During: typing of assignment at <ipython-input-3-290cb8971149> (9)

File "<ipython-input-3-290cb8971149>", line 9:
def f():
    <source elided>
        if i == 999999:
            x = "hello"
            ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should 

In [5]:
import ROOT

ROOT.gInterpreter.Declare("""
int f() {
    int x = 0;
    for (int i = 0; i < 1000000; i++) {
        // in the millionth step, replace x with a string
        if (i == 999999) {
            x = "hello";
        }
        // do an operation on x that only works for integers
        x += 1;
    }
    return x;
}""")

ROOT.f()

AttributeError: f

input_line_33:7:17: error: assigning to 'int' from incompatible type 'const char [6]'
            x = "hello";
                ^~~~~~~
